In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

The function cluster_names will grab the JOB Ids from expanded clusterID.csv file, and merge with job names text file and clean merged database in order to generate an html file and pickle file for a specific cluster. The pickle file holds the pandas dataframe of the cluster without the Project IDs. Thus the html file is the final product which holds the dataframe with user id, job id, project id, and job name.c

In [ ]:
#this is the code to grab the job ids and job names from the clusters
def cluster_names(cluster_num):
    #grabbing the cluster dataframe, this is my specified path but it could be different
    #basically you need the cluster directory folder and the clusterID.csv.gz file (compressed)
    df = pd.read_csv('/Users/cheriehuang/Downloads/Clean_NFS_kmeans/clusterID.csv')
    #grabbing the rows from the cluster you specify with cluster_num
    ndf = df[df['Clusters'] == cluster_num]
    #grabbing the text file with all the jobnames 
    job_names = pd.read_fwf('jobs.txt',names = ['JobID', 'JobNames'])
    stri = 'cluster%d' % cluster_num
    temp_df = ndf.merge(job_names, on = 'JobID', how = 'inner')
    temp_df.to_pickle(stri + '.pkl') #making a pickle file of the dataframe just in case
    #temp_df['JobNames'].value_counts()  -> gives you the frequency of unique names!
    #now grab clean merged
    mdf = pd.read_pickle('clean_merged.pkl')
    #drop the unnecessary columns and make JobID an int
    mdf = mdf[['JobID', 'Project']]
    mdf['JobID'] = mdf['JobID'].astype(int)
    #now merge the two dataframes together
    merged = mdf.merge(temp_df, on = 'JobID', how = 'inner')
    #save this into an html file!
    merged.to_html(stri + '.html')

The code below this markdown is used to open each html file that is given to it, and will generate a pandas dataframe holding the top 5 project ids that occur the most often in that html file. It also will describe the percentage this is of total jobs in that html file.

In [ ]:
names = pd.read_html('DBScanCluster7.html')
#for pandas html files are read in as a list of pandas dataframes so grab the first one
names = names[0]
#counts the frequency of each project ID and puts it in largest to smallest
ldf = names['Project'].value_counts().to_frame(name = 'Count')
#now grab the total number of rows
rows_num = names.shape[0]
#now calculate the percentages for each project ID and put it in the dataframe
ldf['Percentage'] = (ldf['Count']/rows_num) * 100
ldf_h = ldf.head() #grabs the top 5!
ldf_h.to_pickle('DBScanCluster7Top.pkl') #converts it to a pickle
print ldf_h #or you can just print it for a format that is easy to copy and paste directly
